# Linear Model

### Prepare venv

#### const parameters

In [415]:
dir = "../../dataset/variance_big_256/"
labels = ["canter", "trot", "walk"]

rng = 42

batch_size  = 8
epochs      = 10

#### imports

In [ ]:
from IPython.display import clear_output
import numpy as np
import os

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim

#### var parameters

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### load variances

In [417]:
files = sorted(os.listdir(dir))

X_list, y_list = [], []

for label in labels:
    for file in sorted(os.listdir(dir + label)):
        var = np.load(dir + label + "/" + file).astype(np.float32)
        X_list.append(var)
        y_list.append(labels.index(label))

X = np.stack(X_list, axis=0)       # shape: (N, 256, 256)
y = np.array(y_list)               # shape: (N,)

#### train/test split

In [418]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rng)

## Tensors

### Train

In [419]:
X_tensor = torch.tensor(X_train[:, None, :, :], dtype=torch.float32)
y_tensor = torch.tensor(y_train, dtype=torch.long)

dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

### Test

In [420]:
X_test_tensor = torch.tensor(X_test[:, None, :, :], dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Model

In [421]:
class LinearClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(256 * 256, 3)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.fc(x)

## Training

In [430]:
def train(dataLoader, model, criterion, optimizer):
    model.train()
    
    size = len(dataLoader.dataset)
    
    for batch, (X, y) in enumerate (dataLoader):
        X, y = X.to(device), y.to(device)
        
        # Prediction error
        pred = model(X)
        loss = criterion(pred, y)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 25 == 0:
            loss, current = loss.item(), (batch + 1) * batch_size
            print(f"loss: {loss:>7.7f}  \t[{current:>5d}/{size:>5d}]")

In [424]:
def eval(testLoader, model, criterion):
    model.eval()
    
    size = len(testLoader.dataset)
    num_batches = len(testLoader)
    
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for X, y in testLoader:
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += criterion(pred, y)#.item()
            
            correct += (pred.argmax(dim=1) == y).sum().item()
            
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.2f}%, Avg loss: {test_loss:>8.4f} \n")

## Loop
Training is repeated on 10 models to account for randomness

In [ ]:
def calc_confusion_matrix(preds, labels):
    """ rows = true labels, cols = predicted labels """
    cm = torch.zeros((3, 3), dtype=torch.int64)
    for t, p in zip(labels, preds):
        cm[t, p] += 1
    return cm

In [ ]:
def save_results(model, testLoader):
    model.eval()
    
    all_preds, all_labels = [], []

    with torch.no_grad():
        for X, y in testLoader:
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            preds = torch.argmax(pred, dim=1)
            
            all_preds.append(preds)
            all_labels.append(y)

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    cm = calc_confusion_matrix(all_preds, all_labels)
    
    

In [ ]:
for run in range(10):
    
    model = LinearClassifier().to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(
        model.parameters(),
        lr=0.01,
        momentum=0.9)
    
    print("", end="\n\n\n")
    
    for epoch in range(epochs):
    
        train(loader, model, criterion, optimizer)
        clear_output(wait=True)
        eval(test_loader, model, criterion)
        


# Save

In [ ]:
torch.save(model.state_dict(), 'model_temp.pth')